<a href="https://colab.research.google.com/github/life-ofpi/2024_DL/blob/main/24_1_DL_Project_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/DL_term_project')

Mounted at /content/drive


In [2]:
!ls '/content/drive/MyDrive/DL_term_project'

24_1_DL_Project_Training.ipynb	model.py    __pycache__  test.py    train.py
model_param.pth			output.txt  test_dir	 train_dir


In [ ]:
#이것만돌리자
!python /content/drive/MyDrive/DL_term_project/train.py -d '/content/drive/MyDrive/DL_term_project/train_dir' -m '/content/drive/MyDrive/DL_term_project/model_param.pth'

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100% 97.8M/97.8M [00:00<00:00, 158MB/s]
cuda
Epoch 1/20, Loss: 0.7673, Accuracy: 76.11%
Epoch 2/20, Loss: 0.4680, Accuracy: 89.88%
Epoch 3/20, Loss: 0.3623, Accuracy: 91.70%
Epoch 4/20, Loss: 0.3078, Accuracy: 92.38%
Epoch 5/20, Loss: 0.2689, Accuracy: 93.62%
Epoch 6/20, Loss: 0.2320, Accuracy: 94.66%
Epoch 7/20, Loss: 0.2097, Accuracy: 95.15%
Epoch 8/20, Loss: 0.1945, Accuracy: 95.57%
Epoch 9/20, Loss: 0.1801, Accuracy: 96.19%
Epoch 10/20, Loss: 0.1678, Accuracy: 96.35%
Epoch 11/20, Loss: 0.1551, Accuracy: 96.97%
Epoch 12/20, Loss: 0.1477, Accuracy: 96.78%
Epoch 13/20, Loss: 0.1388, Accuracy: 97.23%
Epoch 14/20, Loss: 0.1334, Accuracy: 97.33%
Epoch 15/20, Loss: 0.1243, Accuracy: 97.72%
Epoch 16/20, Loss: 0.1204, Accuracy: 97.40%
Epoch 17/20, Loss: 0.1131, Accuracy: 97.72%
Epoch 18/20, Loss: 0.1075, Accuracy: 97.85%
Epoch 19/20, Loss: 0.1041, Accuracy: 98

In [ ]:
#output 저장
!python /content/drive/MyDrive/DL_term_project/test.py -d '/content/drive/MyDrive/DL_term_project/test_dir' -m '/content/drive/MyDrive/DL_term_project/model_param.pth' -o '/content/drive/MyDrive/DL_term_project/'

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import argparse
from PIL import Image
import os
from model import MyModel
import easydict

'''
parser = argparse.ArgumentParser()
parser.add_argument("-d", "--dataset", type=str, required=True, help="path to input dataset")
parser.add_argument("-m", "--model", type=str, required=True, help="path to model parameter")
args = parser.parse_args()
'''
args = easydict.EasyDict({
    'dataset': '/content/drive/MyDrive/DL_term_project/train_dir',
    'model': '/content/drive/MyDrive/DL_term_project/model_param.pth'
})
root_dir = args.dataset
save_path = args.model

class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths, self.labels = self._get_image_paths_and_labels()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

    def _get_image_paths_and_labels(self):
        image_paths = []
        labels = []
        class_dirs = sorted(os.listdir(self.root_dir))
        for class_index, class_dir in enumerate(class_dirs):
            class_path = os.path.join(self.root_dir, class_dir)
            if os.path.isdir(class_path):
                image_files = sorted(os.listdir(class_path))
                for image_file in image_files:
                    image_path = os.path.join(class_path, image_file)
                    image_paths.append(image_path)
                    labels.append(class_index)
        return image_paths, labels



In [4]:
# model.py
import torch
import torch.nn as nn
from torchvision import models

class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2) # pretrained resnet50 모델 with IMAGENET1K_V2 weights
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.model(x)



In [5]:
#train
transform = transforms.Compose([
    transforms.Resize((224, 224)), # ResNet의 입력 크기에 맞춤
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # ImageNet의 평균과 표준편차로 정규화
])

dataset = ImageDataset(root_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

num_classes = 3
model = MyModel(num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.model.fc.parameters(), lr=0.001)

num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 185MB/s]


cuda


MyModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0

In [6]:
#training accuracy
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0  )
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = correct_predictions / total_predictions
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy*100:.2f}%")

print("Training finished!")

torch.save(model.state_dict(), save_path)
print("Model parameters saved successfully!")

Epoch 1/20, Loss: 0.7821, Accuracy: 74.51%
Epoch 2/20, Loss: 0.4775, Accuracy: 89.16%
Epoch 3/20, Loss: 0.3734, Accuracy: 90.66%
Epoch 4/20, Loss: 0.3041, Accuracy: 92.94%
Epoch 5/20, Loss: 0.2625, Accuracy: 94.37%
Epoch 6/20, Loss: 0.2368, Accuracy: 94.82%
Epoch 7/20, Loss: 0.2170, Accuracy: 95.08%
Epoch 8/20, Loss: 0.1935, Accuracy: 95.41%
Epoch 9/20, Loss: 0.1832, Accuracy: 96.03%
Epoch 10/20, Loss: 0.1719, Accuracy: 96.26%
Epoch 11/20, Loss: 0.1640, Accuracy: 96.09%
Epoch 12/20, Loss: 0.1492, Accuracy: 96.94%
Epoch 13/20, Loss: 0.1348, Accuracy: 97.30%
Epoch 14/20, Loss: 0.1318, Accuracy: 97.49%
Epoch 15/20, Loss: 0.1269, Accuracy: 97.62%
Epoch 16/20, Loss: 0.1231, Accuracy: 97.46%
Epoch 17/20, Loss: 0.1154, Accuracy: 97.59%
Epoch 18/20, Loss: 0.1116, Accuracy: 97.49%
Epoch 19/20, Loss: 0.1064, Accuracy: 97.95%
Epoch 20/20, Loss: 0.0986, Accuracy: 97.82%
Training finished!
Model parameters saved successfully!


In [7]:
#parameter 저장
torch.save(model.state_dict(), save_path)
print("Model parameters saved successfully!")

Model parameters saved successfully!
